# Trabajo Final

###  CLASIFICACION CON MAQUINAS DE SOPORTE VECTORIAL

### Universidad de Antioquia

### Facultad de Ingeniería

### Ingeniería de Sistemas

### Ude@ - 2018-I

#### Profesor: Antonio Tamayo Herrera

### Integrantes:

Nombre: David de Jesus Yepes Isaza

Cédula: 1020455542

Nombre: Rodrigo Antonio Porras Martinez

Cédula: 1039459720

### Clasificación en el dataset "Heart Disease Data Set" con SVM

La base de datos con las que se trabaja en el presente artículo fue extraída de  Cleveland Clinic Foundation y fue creada por V.A. Medical Center, Long Beach and Cleveland Clinic Foundation: Robert Detrano, M.D., Ph.D.

El set de datos de de Cleveland consta de 303 instancias, la variable de salida o etiqueta son 0 para la ausencia y los valores 1, 2, 3 y 4 para la presencia de enfermedades. 
 
El 54% de las instancias tienen ausencia de enfermedades y 46% tienen presencia de enfermedades del corazón. Para simplificar el problema se analiza como un problema biclase donde  “0” son personas sanas y “1” que corresponde a (1,2,3,4) son personas enfermas.

El objetivo es según las características de cada paciente identificar cuál de ellos presenta presencia de enfermedades y cuáles no.

La base de datos contiene 303 muestras con las 14 características siguientes:
1. N° 3 (edad)   
2. N° 4 (sexo) 
3. N° 9 (cp) tipo de dolor en el pecho
        Valor 1: angina típica
        Valor 2: angina atípica
        Valor 3: dolor no anginal
        Valor 4: asintomático  
4. N° 10 (trestbps) presión arterial en reposo
5. N° 12 (chol) colesterol sérico en mg / dl (miligramo por decilitro)
6. N° 16 (fbs) (azúcar en la sangre en ayunas > 120 mg / dl) (1 = verdadero, 0 = falso)
7. N° 19 (restecg)  resultados electrocardiográficos en reposo:
        Valor 0: normal
        Valor 1: tiene anormalidad de onda ST-T (inversiones de onda T y / o ST  elevación o depresión de > 0.05 mV)
        Valor 2: muestra hipertrofia ventricular izquierda probable o definitiva según los criterios de Estes  
8. N° 32 (thalach)  frecuencia cardíaca máxima lograda
9. N° 38 (exang) angina inducida por el ejercicio (1 = sí; 0 = no)
10. N° 40 (oldpeak) depresión del ST inducida por el ejercicio en relación con el reposo
11. N° 41 (pendiente) la pendiente del segmento ST de ejercicio máximo
        Valor 1: pendiente ascendente
        Valor 2: plano
        Valor 3: downsloping
12. N° 44 (ca) número de vasos principales (0-3) coloreados por fluoroscopia
13. N° 51 (thal) 3 = normal; 6 = defecto fijo; 7 = defecto reversible
14. N° 58 (num) (el atributo predicho) diagnóstico de enfermedad cardíaca (estado angiográfico de la enfermedad)
        - Valor 0: <50% de diámetro de estrechamiento
        - Valor 1:> 50% de diámetro de estrechamiento
    

###  Importacion de librerias y limpiado de datos

In [2]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import normalize
from __future__ import division
from sklearn import tree
from sklearn.metrics import confusion_matrix
import time


def class_error(y_est, y_real):
    err = 0
    for y_e, y_r in zip(y_est, y_real):
        if y_e != y_r:
            err += 1

    return "accuracy: " + str(1-(err/np.size(y_est)))

def classification_error(y_est, y_real):
    err = 0
    for y_e, y_r in zip(y_est, y_real):
        if y_e != y_r:
            err += 1
    return err/np.size(y_est)

clevelanddb = np.loadtxt('cleveland.txt',delimiter=',')  # Assuming comma-delimiter
print ("Dim de la base de datos original: " + str(np.shape(clevelanddb)))
DataBase = clevelanddb
#La base de datos cleveland.txt tiene algunos valores perdidos, estos se identifican con -9.0
#Los valores de salida mayores a 0 se considerarán 1 de esta forma la clasificacion será biclase 
j = 0
num_reset_values = 0

for i in range(0,np.size(clevelanddb,0)):
    if -9.0 == clevelanddb[i,13]:
        j+=1
        DataBase = np.delete(DataBase,i,0)
    elif clevelanddb[i,13] > 0:
        clevelanddb[i,13] = 1
        num_reset_values += 1

print ("\nHay " + str(j) + " valores perdidos en la variable de salida.")

print ("\nDim de la base de datos sin las muestras con variable de salida perdido " + str(np.shape(DataBase)))

print ("Se han reseteado a 1 "+str(num_reset_values)+" valores ya que eran mayores a 0")

#Ya hemos eliminado los registros con valor de la variable de salida perdido.

#Ahora vamos a imputar los valores perdidos en cada una de las características
print ("\nProcesando imputación de valores perdidos en las características . . .\n")

for k in range(0,np.size(DataBase,0)):
    for w in range(0,14):
        if -9.0 == DataBase[k,w]:
            DataBase[k,w] = round(np.mean(DataBase[:,w]))
        
print ("Imputación finalizada.\n")

hay_missed_values = False
for i in range(0,np.size(DataBase,0)):
    if -9.0 in DataBase[i,:]:
        hay_missed_values = True
if(hay_missed_values):
    print ("Hay valores perdidos")
else:
    print ("No hay valores perdidos en la base de datos. Ahora se puede procesar. La base de datos está en la variable DataBase")


Dim de la base de datos original: (303, 14)

Hay 0 valores perdidos en la variable de salida.

Dim de la base de datos sin las muestras con variable de salida perdido (303, 14)
Se han reseteado a 1 139 valores ya que eran mayores a 0

Procesando imputación de valores perdidos en las características . . .

Imputación finalizada.

No hay valores perdidos en la base de datos. Ahora se puede procesar. La base de datos está en la variable DataBase


### Aplicando SVM con 10 folds

In [3]:
from sklearn import svm

X = DataBase[:,0:13]
Y = DataBase[:,13]


#Normalizamos los datos
#X = normalize(X, axis=1, norm='l1')

#Implemetamos la metodología de validación cross validation con 10 folds

#Para calcular el costo computacional
tiempo_i = time.time()

Sensibilidad = np.ones(10)
Especificidad = np.ones(10)
Precision = np.ones(10)
Eficiencia = np.ones(10)
Errores = np.ones(10) 

j = 0
fold_num = 0
kf = KFold(n_splits=10)

#Complete el código para crear el clasificador SVM. Tenga en cuenta que el problema es multiclase. 
#Consulte como lo puede tratar con la SVM.
#Use un kernel rbf con una malla de valores así: C en {0.1,1,10,100} y gamma en {0.0001,0.01, 0.1}
clf = svm.SVC(C=100, gamma=0.0001, kernel='rbf')

for train_index, test_index in kf.split(X):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]  
    fold_num += 1
    
    model = clf.fit(X_train, y_train) 
    
    y_pred = model.predict(X_test)
    
    tn, fp, fn, tp = confusion_matrix(y_test,y_pred ).ravel()
    
    Errores[j] = classification_error(y_pred, y_test)
    Sensibilidad[j] = tp/(tp+fn)
    Especificidad[j] = tn/(tn+fp)
    Precision[j] = tp/(tp+fp)
    Eficiencia[j] = (tp+tn)/(tp+tn+fp+fn)
    j+=1
    
print("\nError de validación: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))
print("\nSensibilidad: "+str(np.mean(Sensibilidad)))
print("\nEspecificidad: "+str(np.mean(Especificidad)))
print("\nPrecision: "+str(np.mean(Precision)))
print("\nEficiencia : "+str(np.mean(Eficiencia)) + "\n ")
print ("\n\nTiempo total de ejecución: " + str(time.time()-tiempo_i) + " segundos.")





Error de validación: 0.20150537634408602 +/- 0.05492340076360415

Sensibilidad: 0.7574175579322638

Especificidad: 0.838938154209052

Precision: 0.798997668997669

Eficiencia : 0.798494623655914
 


Tiempo total de ejecución: 0.105841159821 segundos.


 <table>
  <tr>
    <th>C</th>
    <th>GAMMA</th>
    <th>ERROR DE VALIDACION</th>
    <th>SENSIBILIDAD</th>
    <th>ESPECIFICIDAD</th>
    <th>PRECISION</th>
    <th>EFICIENCIA</th>
    <th>TIEMPO TOTAL DE EJCUCION</th>
    
  </tr>
  <tr>
    <td>0.1</td>
    <td>0.0001</td>
    <td>0.3791397849462366 +/- 0.08705454970106913</td>
    <td>0.4007916348357525</td>
    <td>0.8137690663309858</td>
    <td>0.6641913641913642</td>
    <td>0.6208602150537634</td>
    <td>0.0542011260986 segundos.</td>
  </tr>
  <tr>
    <td>0.1</td>
    <td>0.01</td>
    <td>0.4590322580645161 +/- 0.07007989758356642</td>
    <td>0.0</td>
    <td>1.0</td>
    <td>nan</td>
    <td>0.5409677419354839</td>
    <td>0.0644090175629 segundos.</td>
  </tr>
  <tr>
    <td>0.1</td>
    <td>0.1</td>
    <td>0.4590322580645161 +/- 0.07007989758356642</td>
    <td>0.0</td>
    <td>1.0</td>
    <td>nan</td>
    <td>0.5409677419354839</td>
    <td>0.0751130580902 segundos.</td>
  </tr>
  
  <tr>
    <td>1</td>
    <td>0.0001</td>
    <td>0.3364516129032258 +/- 0.07612941194243453</td>
    <td>0.5466418385536033</td>
    <td>0.7650406134113564</td>
    <td>0.6736796536796537</td>
    <td>0.6635483870967741</td>
    <td>0.0499041080475 segundos.</td>
  </tr>
  <tr>
    <td>1</td>
    <td>0.01</td>
    <td>0.4294623655913978 +/- 0.0995064708952584</td>
    <td>0.32879933791698496</td>
    <td>0.7853476309552161</td>
    <td>0.5590151515151515</td>
    <td>0.5705376344086022</td>
    <td>0.0750560760498 segundos.</td>
  </tr>
  <tr>
    <td>1</td>
    <td>0.1</td>
    <td>0.4590322580645161 +/- 0.07007989758356642</td>
    <td>0.0</td>
    <td>1.0</td>
    <td>nan</td>
    <td>0.5409677419354839</td>
    <td>0.0773460865021 segundos.</td>
  </tr>
  
  <tr>
    <td>10</td>
    <td>0.0001</td>
    <td>0.26731182795698927 +/- 0.06862068715083379</td>
    <td>0.6391539343009931</td>
    <td>0.8189882795223353</td>
    <td>0.7484737484737486</td>
    <td>0.7326881720430107</td>
    <td>0.0536198616028 segundos.</td>
  </tr>
  <tr>
    <td>10</td>
    <td>0.01</td>
    <td>0.4327956989247312 +/- 0.11298026045771377</td>
    <td>0.4161911128087598</td>
    <td>0.7029623408067681</td>
    <td>0.5437229437229436</td>
    <td>0.5672043010752688</td>
    <td>0.0783281326294 segundos.</td>
  </tr>
  <tr>
    <td>10</td>
    <td>0.1</td>
    <td>0.45258064516129026 +/- 0.07476397516777847</td>
    <td>0.01575757575757576</td>
    <td>1.0</td>
    <td>nan</td>
    <td>0.5474193548387097</td>
    <td>0.0833170413971 segundos.</td>
  </tr>
  <tr>
    <td>100</td>
    <td>0.0001</td>
    <td>0.20150537634408602 +/- 0.05492340076360415</td>
    <td>0.7574175579322638</td>
    <td>0.838938154209052</td>
    <td>0.798997668997669</td>
    <td>0.798494623655914</td>
    <td>0.0930299758911 segundos.</td>
  </tr>
  <tr>
    <td>100</td>
    <td>0.01</td>
    <td>0.4327956989247312 +/- 0.11298026045771377</td>
    <td>0.4161911128087598</td>
    <td>0.7029623408067681</td>
    <td>0.5437229437229436</td>
    <td>0.5672043010752688</td>
    <td>0.0818700790405 segundos.</td>
  </tr>
  <tr>
    <td>100</td>
    <td>0.1</td>
    <td>0.45258064516129026 +/- 0.07476397516777847</td>
    <td>0.01575757575757576</td>
    <td>1.0</td>
    <td>nan</td>
    <td>0.5474193548387097</td>
    <td>0.0846180915833 segundos.</td>
  </tr>
</table>

## Selección de Caracteristicas - Wrappers

Para selecionar las caracteristicas usaremos la funcion Wrappers y como objetivo MLP. los parámetros son el número de características a seleccionar  (3, 7 y 10) y la estrategia a implementar (SFS, SBS, SFFS, SBFS)

In [24]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.neighbors import KNeighborsClassifier

#Feature Selection Function
#Recibe 4 parámetros: 1. el modelo (clf para nuestro caso), 2. el número de características final que se quiere alcanzar
#3. Si es forward (True), si es Backward False, 4. Si es es flotante (True), sino False
#SFS True False, SBS False False, SFFS True True, SBFS False True
m_forward = False
m_floating = True
m_features = 7

scores =['accuracy', 'precision', 'recall']

def select_features(modelo, n_features, fwd, fltg,score):
    
    sfs = SFS(modelo, 
           k_features=n_features, 
           forward=fwd,
           floating=fltg,
           verbose=1,
           scoring = score,
           cv=0)
    
    return sfs


#Para calcular el costo computacional
tiempo_i = time.time()

#Implemetamos la metodología de validación cross validation con 10 folds

Medidas = np.ones(10)
MedidasPromedio = np.ones(3)
k = 0

j = 0
kf = KFold(n_splits=10)

#Mejores parametros c= 100 y gamma = 0.0001
clf = svm.SVC(C=100, gamma=0.0001, kernel='rbf')
for scr in scores:
    for train_index, test_index in kf.split(X):

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]  

        #Aquí se entrena y se valida el modelo haciendo selección de características con diferentes estrategias

        #Complete el código llamando el método select_features con los parámetros correspondientes para responder el
        #Ejercicio 3.1
        #SFS True False, SBS False False, SFFS True True, SBFS False True

        sf = select_features(clf,m_features,m_forward,m_floating,scr)

        #Complete el código para entrenar el modelo con las características seleccionadas. Tenga en cuenta
        #la metodología de validación aplicada para que pase las muestras de entrenamiento correctamente.
        sf = sf.fit(X_train,y_train)
        
        Medidas[j] = sf.k_score_
        j+=1
        
    MedidasPromedio[k]=np.mean(Medidas)
    k+=1
    j=0



print("SFS = True False, SBS = False False, SFFS = True True, SBFS = False True")
print (str(m_forward)+"-"+str(m_floating) )
print("# Caracteristicas a elegir: "+str(m_features))
print("\nError en validación : " + str((1-MedidasPromedio[0])*100) + "%" )
print("\nEficiencia : " + str(MedidasPromedio[0]*100) + "%" )
print("\nPresicion : " + str(MedidasPromedio[1]*100) + "%" )
print("\nSensibilidad : " + str(MedidasPromedio[2]*100) + "%" )

print ("\n\nTiempo total de ejecución: " + str(time.time()-tiempo_i) + " segundos.")

print("Las caracteristicas seleccionadas son: ")

dictChar = {0 : "1. edad", 
            1 : '2. sexo',
            2 : '3. (cp) tipo de dolor en el pecho',
            3 : '4. (trestbps) presión arterial en reposo',
            4 : '5. (chol) colesterol sérico en mg / dl (miligramo por decilitro)',
            5 : '6. (fbs) (azúcar en la sangre en ayunas > 120 mg / dl) (1 = verdadero, 0 = falso)',
            6 : '7. (restecg) resultados electrocardiográficos en reposo:',
            7 : '8. (thalach) frecuencia cardíaca máxima lograda',
            8 : '9. (exang) angina inducida por el ejercicio (1 = sí; 0 = no)',
            9 : '10. (oldpeak) depresión del ST inducida por el ejercicio en relación con el reposo',
            10 :'11. (pendiente) la pendiente del segmento ST de ejercicio máximo',
            11 :'12. (ca) número de vasos principales (0-3) coloreados por fluoroscopia',
            12 :'13. (thal) 3 = normal; 6 = defecto fijo; 7 = defecto reversible'}

for x in range(0,len(sf.k_feature_idx_)):
    print (dictChar[sf.k_feature_idx_[x]])

[Parallel(n_jobs=1)]: Done  13 out of  13 | elapsed:    0.1s finished
Features: 12/7[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
Features: 11/7[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished
Features: 10/7[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
Features: 9/7[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s finished
Features: 8/7[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s finished
Features: 7/7[Parallel(n_jobs=1)]: Done  13 out of  13 | elapsed:    0.1s finished
Features: 12/7[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.1s fini

Features: 9/7[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s finished
Features: 8/7[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished
Features: 7/7[Parallel(n_jobs=1)]: Done  13 out of  13 | elapsed:    0.1s finished
Features: 12/7[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
Features: 11/7[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished
Features: 10/7[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
Features: 9/7[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s finis

Features: 10/7[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
Features: 9/7[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s finished
Features: 8/7[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s finished
Features: 7/7[Parallel(n_jobs=1)]: Done  13 out of  13 | elapsed:    0.1s finished
Features: 12/7[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
Features: 11/7[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished
Features: 10/7[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s fini

Features: 8/7[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s finished
Features: 7/7[Parallel(n_jobs=1)]: Done  13 out of  13 | elapsed:    0.1s finished
Features: 12/7[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
Features: 11/7[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished
Features: 10/7[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
Features: 9/7[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s finished
Features: 8/7[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finis

SFS = True False, SBS = False False, SFFS = True True, SBFS = False True
False-True
# Caracteristicas a elegir: 7

Error en validación : 15.438887093298847%

Eficiencia : 84.56111290670115%

Presicion : 90.16216351049366%

Sensibilidad : 82.01360771551596%


Tiempo total de ejecución: 27.6423211098 segundos.
Las caracteristicas seleccionadas son: 
1. edad
4. (trestbps) presión arterial en reposo
5. (chol) colesterol sérico en mg / dl (miligramo por decilitro)
8. (thalach) frecuencia cardíaca máxima lograda
10. (oldpeak) depresión del ST inducida por el ejercicio en relación con el reposo
12. (ca) número de vasos principales (0-3) coloreados por fluoroscopia
13. (thal) 3 = normal; 6 = defecto fijo; 7 = defecto reversible



<table>
  <tr>
    <th># FEATURES</th>
    <th>TECNICA</th>
    <th>ERROR</th>
    <th>EFICIENCIA </th>
    <td>PRECISION</td>
    <td>SENSIBILIDAD</td>
    <td>TIEMPO DE EJECUCION(S)</td>
  </tr>
  <tr>
    <td>3</td>
    <td>SVM + SFS</td>
    <td>20.682369101486753%</td>
    <td>79.31763089851324%</td>
    <td>82.90299755460788%</td>
    <td>82.44875833000134%</td>
    <td>8.15271615982</td>
 </tr>
 <tr>
    <td>3</td>
    <td>SVM + SBS</td>
    <td>21.709491488903264%</td>
    <td>78.29050851109673%</td>
    <td>84.21734357963527%</td>
    <td>78.89537514026647%</td>
    <td>28.1594088078</td>
 </tr>
  <tr>
    <td>3</td>
    <td>SVM + SFFS</td>
    <td>20.682369101486753%</td>
    <td>79.31763089851324%</td>
    <td>83.73126181265145%</td>
    <td>83.32875833000134%</td>
    <td>9.2335870266</td>
 </tr>
  <tr>
    <td>3</td>
    <td>SVM + SBFS</td>
    <td>19.58373734109028%</td>
    <td>80.41626265890972%</td>
    <td>86.39193463612601%</td>
    <td>78.89537514026647%</td>
    <td>53.4099361897</td>
 </tr>
 <tr>
    <td>5</td>
    <td>SVM + SFS</td>
    <td>16.794871794871803%</td>
    <td>83.2051282051282%</td>
    <td>86.12327101634844%</td>
    <td>81.78703033571516%</td>
    <td>13.8639409542</td>
 </tr>
 <tr>
    <td>5</td>
    <td>SVM + SBS</td>
    <td>18.62920168067227%</td>
    <td>81.37079831932773%</td>
    <td>86.93770691529103%</td>
    <td>80.88693994713942%</td>
    <td>24.3941957951</td>
 </tr>
  <tr>
    <td>5</td>
    <td>SVM + SFFS</td>
    <td>16.794871794871803%</td>
    <td>83.2051282051282%</td>
    <td>86.73188501561881%</td>
    <td>83.12078033571517%</td>
    <td>16.3676300049</td>
 </tr>
  <tr>
    <td>5</td>
    <td>SVM + SBFS</td>
    <td>18.153011204481796%</td>
    <td>81.8469887955182%</td>
    <td>87.87560420114498%</td>
    <td>80.88693994713942%</td>
    <td>39.0618109703</td>
 </tr>
 <tr>
    <td>7</td>
    <td>SVM + SFS</td>
    <td>15.989010989010987%</td>
    <td>84.01098901098901%</td>
    <td>86.9946443181528%</td>
    <td>78.85720115179736%</td>
    <td>16.9370970726</td>
 </tr>
 <tr>
    <td>7</td>
    <td>SVM + SBS</td>
    <td>15.438887093298847%</td>
    <td>84.56111290670115%</td>
    <td>90.12520114490228%</td>
    <td>82.01360771551596%</td>
    <td>21.6310710907</td>
 </tr>
  <tr>
    <td>7</td>
    <td>SVM + SFFS</td>
    <td>15.989010989010987%</td>
    <td>84.01098901098901%</td>
    <td>87.95770997664297%</td>
    <td>78.45901566792637%</td>
    <td>22.7561228275</td>
 </tr>
  <tr>
    <td>7</td>
    <td>SVM + SBFS</td>
    <td>15.438887093298847%</td>
    <td>84.56111290670115%</td>
    <td>90.16216351049366%</td>
    <td>82.01360771551596%</td>
    <td>27.6423211098</td>
 </tr>
</table>

In [31]:
from mlxtend.preprocessing import standardize
from mlxtend.feature_extraction import PrincipalComponentAnalysis as PCA

#Feature Extraction Function
#Recibe 2 parámetros: 1. el tipo de método de extracción (pca o lda como string), 2. el número componentes (para pca)
#o el número de discriminantes (para lda)

def extract_features(tipo, n):
    
    if tipo == 'pca':
    
        ext = PCA(n_components=n)
    
        return ext

    elif tipo == 'lda':
        
        ext = LDA(n_discriminants=n)
        
        return ext
    
    else:
        print ("Ingrese un método válido (pca o lda)\n")

#Para calcular el costo computacional
tiempo_i = time.time()

X = DataBase[:,0:13]
#Estandarizamos los datos
X = standardize(X)

#Implemetamos la metodología de validación cross validation con 10 folds

Errores = np.ones(10)
Sensibilidad = np.ones(10)
Especificidad = np.ones(10)
Precision = np.ones(10)
Eficiencia = np.ones(10)

j = 0
kf = KFold(n_splits=10)
#Mejores parametros c= 100 y gamma = 0.0001
clf = svm.SVC(C=100, gamma=0.0001, kernel='rbf')

for train_index, test_index in kf.split(X):
    
    #Aquí se aplica la extracción de características por PCA
    ex = extract_features('pca',12) 

    #Fit de PCA
    ex = ex.fit(X)
    
    #Transforme las variables y genere el nuevo espacio de características de menor dimensión
    X_ex = ex.transform(X)
    
    #Se aplica CV-10
    
    X_train, X_test = X_ex[train_index], X_ex[test_index]
    y_train, y_test = Y[train_index], Y[test_index]  
    
    # Entrenamiento el modelo.
    model = clf.fit(X_train,y_train)

    # Validación del modelo
    ypred = model.predict(X_test)

    tn, fp, fn, tp = confusion_matrix(y_test,ypred).ravel()

    Errores[j] = classification_error(ypred, y_test)
    Sensibilidad[j] = tp/(tp+fn)
    Especificidad[j] = tn/(tn+fp)
    Precision[j] = tp/(tp+fp)
    Eficiencia[j] = (tp+tn)/(tp+tn+fp+fn)
    j+=1
        

print("\nError de validación: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))       
print("\nSensibilidad: "+str(np.mean(Sensibilidad)))
print("\nEspecificidad: "+str(np.mean(Especificidad)))
print("\nPrecision: "+str(np.mean(Precision)))
print("\nEficiencia : "+str(np.mean(Eficiencia)) + "\n ")
print ("\n\nTiempo total de ejecución: " + str(time.time()-tiempo_i) + " segundos.")




Error de validación: 0.1586021505376344 +/- 0.08155649941501905

Sensibilidad: 0.7855665902724727

Especificidad: 0.8910861713106295

Precision: 0.8625420193841247

Eficiencia : 0.8413978494623656
 


Tiempo total de ejecución: 0.0574591159821 segundos.



<table>
  <tr>
    <th># COMPONENTES</th>
    <th>TECNICA</th>
    <th>ERROR</th>
     <td>SENSIBILIDAD</td>
     <td>ESPECIFICIDAD</td>
      <td>PRECISION</td>
    <th>EFICIENCIA </th>
    <td>TIEMPO DE EJECUCION(S)</td>
  </tr>
  <tr>
    <td>2</td>
    <td>SVM + PCA</td>
    <td>0.27408602150537636 +/- 0.08680483396624582</td>
    <td>0.6269139928698753</td>
    <td>0.8112728370700507</td>
    <td>0.7434706959706958</td>
    <td>0.7259139784946236</td>
    <td>0.0484209060669</td>
    
 </tr>
 <tr>
    <td>6</td>
    <td>SVM + PCA</td>
    <td>0.21795698924731183 +/- 0.09398741376440219</td>
    <td>0.724451871657754</td>
    <td>0.8335974977035349</td>
    <td>0.7853712953712954</td>
    <td>0.7820430107526881</td>
    <td>0.0576889514923</td>
 </tr>
    <tr>
    <td>10</td>
    <td>SVM + PCA</td>
    <td>0.1813978494623656 +/- 0.07264083525765849</td>
    <td>0.7649605296664121</td>
    <td>0.857999027546241</td>
    <td>0.8243653568653568</td>
    <td>0.8186021505376344</td>
    <td>0.0441558361053</td>
 </tr>
    <tr>
    <td>12</td>
    <td>SVM + PCA</td>
    <td>0.15526881720430108 +/- 0.07907752342813566</td>
    <td>0.7855665902724727</td>
    <td>0.8963493292053663</td>
    <td>0.8686026254447308</td>
    <td>0.8447311827956989</td>
    <td>0.0755228996277</td>
 </tr>
</table>